In [ ]:
# Install medcat
! pip install medcat==1.2.3

In [ ]:
from tokenizers import ByteLevelBPETokenizer
import gensim
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

In [ ]:
DATA_DIR = "./data/"

In [ ]:
!mkdir ./data
!mkdir ./models
!wget https://raw.githubusercontent.com/CogStack/MedCATtutorials/main/notebooks/introductory/data/noteevents.csv -P ./data/

--2021-10-24 20:08:30--  https://raw.githubusercontent.com/CogStack/MedCAT/master/tutorial/data/noteevents.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7171226 (6.8M) [text/plain]
Saving to: ‘./data/noteevents.csv’

noteevents.csv      100%[===================>]   6.84M  --.-KB/s    in 0.08s   

2021-10-24 20:08:30 (84.4 MB/s) - ‘./data/noteevents.csv’ saved [7171226/7171226]



### Meta Annotations with MedCAT

To train meta-annotations (e.g. Experiencer, Negation...) we need two additional models:
- Tokenizer: to tokenize the text
- Embeddings: Word2Vec or any other type of embeddings that will be used for meta annotations. 

For meta-annotations we will use a custom BiLSTM model with simulated attention that works very well with sub-word tokenizers and embeddings creating using Word2Vec or BERT (for simplicity we will use w2v here). All of this is also available for download (check next tutorial) and we only need to rebuild the tokenizer/embeddings if our use-case is from a very specific domain. 

In [ ]:
# To train the tokenizer we will use all the data we have from our dummy dataset.
df = pd.read_csv(DATA_DIR + "noteevents.csv")
df.head()

,Unnamed: 0,subject_id,chartdate,category,text
0,0,0,01/01/2086,Urology,"CHIEF COMPLAINT: , Blood in urine.,HISTORY OF ..."
1,1,0,01/01/2086,Emergency Room Reports,"CHIEF COMPLAINT: , Blood in urine.,HISTORY OF ..."
2,2,0,01/01/2086,General Medicine,"CHIEF COMPLAINT: , Blood in urine.,HISTORY OF ..."
3,3,0,01/01/2086,General Medicine,"CHIEF COMPLAINT:, Followup on hypertension an..."
4,4,0,01/01/2086,Consult - History and Phy.,"CHIEF COMPLAINT: , Blood in urine.,HISTORY OF ..."


In [ ]:
# The tokenizers from huggingface require us to save all the text used for 
#training into one/multiple text files.
f = open(DATA_DIR + "tok_data.txt", 'w')
for text in df['text'].values:
  #We'll remove new lines, so that we have one document in one line
  text = text.strip().replace("\n", ' ')
  f.write(text.lower()) # Lowercase text to remove noise
  f.write("\n")
f.close()

In [ ]:
# Create, train and save the tokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(DATA_DIR + "tok_data.txt")
tokenizer.save("./models/bbpe")

In [ ]:
# Now we tokenize all the text we have and train word2vec
f = open(DATA_DIR + "tok_data.txt", 'r')
# Note that if you have a very large dataset, use iterators that
#read the text line by line from the file, do not load the whole file
#into memory.
data = []
for line in f:
  data.append(tokenizer.encode(line).tokens)
w2v = Word2Vec(data, vector_size=300, min_count=1)

In [ ]:
# Check is word2vec trained, Ġ - for this tokenizer denotes start of word (a space)
w2v.wv.most_similar('Ġcancer')

[('Ġcolon', 0.7221731543540955),
 ('Ġcarcinoma', 0.7145481705665588),
 ('Ġmetastatic', 0.6969555616378784),
 ('Ġmesothelioma', 0.6846612095832825),
 ('Ġfather', 0.6737938523292542),
 ('Ġdied', 0.6339669823646545),
 ('Ġmother', 0.6332935094833374),
 ('Ġlung', 0.6187140345573425),
 ('Ġbreast', 0.6179347038269043),
 ('Ġfamily', 0.6172316074371338)]

In [ ]:
# Now we just have to create the embeddings matrix
embeddings = []
for i in range(tokenizer.get_vocab_size()):
  word = tokenizer.id_to_token(i)
  if word in w2v.wv:
    embeddings.append(w2v.wv[word])
  else:
    # Assign a random vector if the word was not frequent enough to receive
    #an embedding
    embeddings.append(np.random.rand(300))

In [ ]:
# Save the embeddings
np.save(open("./models/embeddings.npy", 'wb'), np.array(embeddings))